# 使用PAI预置算法微调模型

预训练模型（pre-trained model）是通过在大规模数据集上进行训练，从而学习到数据的特征表示的深度学习模型。因为模型是通过大规模的数据进行预训练，因而可以通过少量的数据集进行训练，避免从头训练模型的高额成本。在应用时，预训练模型可以被作为基础模型，然后在特定任务的有标注数据集上进行微调，从而适应特定任务的要求。

PAI在公共模型仓库中，提供了不同领域，包括计算机视觉、自然语言处理、语音等的常见热门预训练模型，例如 `QWen`、`Bert`、`ChatGLM`、`LLama2`、`StableDiffusion 2.1` 等，并提供了相应的预置算法，用户仅需提供数据集，即可在PAI上完成模型微调训练。

在本示例中，我们将以`Bert`模型为示例，展示如何使用PAI提供的预置算法对模型进行微调训练。


## 费用说明

本示例将会使用以下云产品，并产生相应的费用账单：

- PAI-DLC：运行训练任务，详细计费说明请参考[PAI-DLC计费说明](https://help.aliyun.com/zh/pai/product-overview/billing-of-dlc)
- PAI-EAS：部署推理服务，详细计费说明请参考[PAI-EAS计费说明](https://help.aliyun.com/zh/pai/product-overview/billing-of-eas)
- OSS：存储训练任务输出的模型、训练代码、TensorBoard日志等，详细计费说明请参考[OSS计费概述](https://help.aliyun.com/zh/oss/product-overview/billing-overview)


> 通过参与云产品免费试用，使用**指定资源机型**提交训练作业或是部署推理服务，可以免费试用PAI产品，具体请参考[PAI免费试用](https://help.aliyun.com/zh/pai/product-overview/free-quota-for-new-users)。




## 安装和配置SDK


我们需要首先安装PAI Python SDK以运行本示例。

In [ ]:

!python -m pip install --upgrade pai



SDK需要配置访问阿里云服务需要的AccessKey，以及当前使用的工作空间和OSS Bucket。在PAI SDK安装之后，通过在**命令行终端** 中执行以下命令，按照引导配置密钥、工作空间等信息。


```shell

# 以下命令，请在 命令行终端 中执行.

python -m pai.toolkit.config

```

我们可以通过以下代码验证配置是否已生效。

In [ ]:
import pai
from pai.session import get_default_session

print(pai.__version__)

sess = get_default_session()

# 获取配置的工作空间信息
assert sess.workspace_name is not None
print(sess.workspace_name)

## 查看PAI提供的预训练模型

我们可以通过参数`provider`为`pai`，获取`PAI`公共模型仓库下的模型，其中包含了PAI提供的模型和从开源社区精选的模型。


In [6]:
from pai.model import RegisteredModel


data = [["ModelName", "Task", "Revision"]]

# 获取公共模型仓库'pai'提供的"大语言"模型列表
for m in RegisteredModel.list(model_provider="pai", task="large-language-model"):
    revision = m.version_labels.get("revision")
    license = m.version_labels.get("license")
    task = m.task
    data.append([m.model_name, task, revision])

In [7]:
from IPython.display import HTML, display

display(
    HTML(
        "<table><tr>{}</tr></table>".format(
            "</tr><tr>".join(
                "<td>{}</td>".format("</td><td>".join(str(_) for _ in row))
                for row in data
            )
        )
    )
)

ModelName,Task,Revision
qwen1.5-1.8b-chat,large-language-model,None
qwen1.5-0.5b,large-language-model,None
qwen1.5-0.5b-chat,large-language-model,None
chatglm3-6b,large-language-model,None
sqlcoder-7b,large-language-model,None
qwen-1_8b-chat,large-language-model,None
baichuan2-7b-chat,large-language-model,None
baichuan2-7b-base,large-language-model,None
qwen-7b-chat,large-language-model,None
qwen-7b,large-language-model,None


## 使用模型的预置算法微调训练

通过`model_name`和`model_provider`参数，我们可以获取PAI提供的预训练模型(`RegisteredModel`对象)，`RegisteredModel`对象包含了模型所在的OSS Bucket信息，以及模型的预训练算法配置。

In [ ]:
from pai.model import RegisteredModel

# 获取PAI模型仓库中的bert-base-uncased模型
m = RegisteredModel(
    model_name="bert-base-uncased",
    model_provider="pai",
)

print(m)

# 查看模型的公共读OSS Bucket路径
print(m.model_data)
# 查看模型的训练算法配置
print(m.training_spec)

获取 `bert-base-uncased` 模型的预置微调算法，以及算法的超参定义和输入数据定义。

In [ ]:
import json
from pai.estimator import AlgorithmEstimator


# 通过注册模型的配置，获取相应的预训练算法
est: AlgorithmEstimator = m.get_estimator()

# 查看算法的超参定义
print(json.dumps(est.hyperparameter_definitions, indent=4))

In [ ]:
# 查看算法的输入输出通道定义
print(json.dumps(est.input_channel_definitions, indent=4))

In [ ]:
# 默认的超参信息
print("before")
print(est.hyperparameters)


# 配置超参
est.set_hyperparameters(batch_size=32)

print("after")
print(est.hyperparameters)

模型默认自带了测试的训练数据，例如BERT模型提供的预置算法可以用于训练一个文本分类模型，默认提供了[情感分类数据集sst2](https://huggingface.co/datasets/sst2)，可以直接用于模型的微调训练。
训练数据格式为一个`jsonline`文件，每一行为一个json对象，包含了`label`和`text`两个字段，分别表示文本的标签和文本内容。

```json
{"label": "negative", "text": "hide new secretions from the parental units "}
{"label": "negative", "text": "contains no wit , only labored gags "}
{"label": "positive", "text": "that loves its characters and communicates something rather beautiful about human nature "}
...
...

```

用户可以参考以上的数据格式准备数据，当前示例中，我们将基于PAI提供的数据集对模型进行微调训练。

In [ ]:
# 获取模型自带的训练输入
default_inputs = m.get_estimator_inputs()

# 默认的算法训练输入，包含了算法使用的预训练模型，训练数据，以及验证数据。
print(json.dumps(default_inputs, indent=4))

我们将模型配置的默认的数据集作为训练输入，使用模型预置的PAI算法提交训练作业。

In [ ]:
est.fit(inputs=default_inputs)

在训练结束之后获取产出模型的OSS Bucket路径。

In [ ]:
# 查看输出模型路径
print(est.model_data())